<a href="https://colab.research.google.com/github/Bk073/Word2Vec/blob/main/Word2Vec_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import io
import re
import string
import tqdm

import tensorflow as tf
from tensorflow.keras import layers

import random
from collections import Counter
import random
import numpy as np
import itertools

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

### Process data

In [ ]:
def read_file(file_path):
  import codecs
  with codecs.open(file_path, 'r', encoding='latin1',errors='ignore') as fdata:
    data = fdata.read()
  return data
        
  # with open(file_path, 'r') as f:
  #   for line in f:
  #     line = bytes(line, 'utf-8').decode('utf-8', 'ignore')
  #     content += line
  # return content

In [ ]:
# Download 20-news group data
# file_path = tf.keras.utils.get_file(fname='wiki_dump', origin='http://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2', extract=True, cache_subdir='/content/drive/MyDrive/word2vec')

In [ ]:
# import tarfile

# t_file = tarfile.open('/content/drive/MyDrive/word2vec/20_news_group.tar.gz')
# t_file.extractall('/content/drive/MyDrive/word2vec/20_news_group')
# t_file.close()

In [ ]:
def data_dump(root_path):
  train_data_list = ''
  for i in os.listdir(root_path):
    f = os.path.join(root_path, i)
    for j in os.listdir(f):
      # train_data_list.append(os.path.join(f, j))
      train_data_list += read_file(os.path.join(f, j))
      # except:
        # import pdb
        # pdb.set_trace()
  
  return train_data_list

In [ ]:
# train_dir = '/content/drive/MyDrive/word2vec/20_news_group/20news-bydate-train'
# word_list = data_dump(train_dir)

In [ ]:
# with open('/content/drive/MyDrive/word2vec/words.txt', 'w') as f:
#   f.write(word_list)

In [5]:
raw_file_path =  '/content/drive/MyDrive/word2vec/words.txt'
with open(raw_file_path, 'r') as f:
  raw_file = f.read()

In [47]:
def preprocess_text(sentence, lang):
  # lower = sentence.lower()
  # email_removed = tf.strings.regex_replace(
  #     lower, "\S*@\S*\s?", ""
  # )
  # number_removed = tf.strings.regex_replace(
  #     email_removed, "[0-9]", ' '
  # )
  # punctuation_removed =  tf.strings.regex_replace(
  #     number_removed, '[%s]' % re.escape(string.punctuation), ' '
  # )
  # multiple_space_removed =tf.strings.reduce_join(tf.strings.split(punctuation_removed), separator=" ")
  # return multiple_space_removed
  if lang == 'eng':
    punctuation_ = set(string.punctuation)
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    sentence = sentence.lower()
    sentence = re.sub(r'\S*@\S*\s?', ' ', sentence)
    sentence = re.sub(r'[0-9]', ' ', sentence)
    # sentence = ''.join(ch for ch in sentence if ch not in punctuation_)
    # sentence = re.sub(r'[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+\ *', ' ', sentence)
    # sentence = re.sub(r"[!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+\ *", " ", sentence)
    sentence = sentence.translate(translator)
    sentence = re.sub(r'\n', ' ', sentence)
    sentence = re.sub(r'\t+', ' ', sentence)
    sentence = re.sub(' +', ' ', sentence)
    sentence = sentence.strip()
    return sentence



In [48]:
preprocessed_data = preprocess_text(raw_file, 'eng')
print(preprocessed_data[:100])

from mathew subject alt atheism faq atheist resources summary books addresses music anything related


In [120]:
def subsampling(train_data, threshold=1e-5):
  total_words = train_data.split(' ')
  print(f"total words: {len(total_words)}")
  print(f"unique words: {len(set(total_words))}")

  c = Counter (total_words)
  total_counts = len(total_words)
  freq = {word: (count / total_counts) for word, count in c.items()}
  probability = {word: (1-np.sqrt(threshold/freq[word])) for word in c}

  train_words = [word for word in total_words if 0.2 < (1 - probability[word])]
  print(f"subsampled vocab length: {len(train_words)}")

  return train_words


In [121]:
subsampled_words = subsampling(preprocessed_data)

total words: 3300978
unique words: 77710
subsampled vocab length: 1245470


In [122]:
subsampled_words[:20]

['mathew',
 'alt',
 'atheism',
 'faq',
 'atheist',
 'resources',
 'summary',
 'books',
 'addresses',
 'music',
 'related',
 'atheism',
 'faq',
 'atheism',
 'books',
 'music',
 'fiction',
 'addresses',
 'contacts',
 'expires']

In [30]:
train_data = tf.data.TextLineDataset(raw_file_path)

In [31]:
for text in train_data.take(2):
  print(text.numpy())

b'From: mathew <mathew@mantis.co.uk>'
b'Subject: Alt.Atheism FAQ: Atheist Resources'


In [77]:
def preprocess(text):
  lower = tf.strings.lower(text)
  # remove emails
  email_removed = tf.strings.regex_replace(
      lower, "\S*@\S*\s?", ""
  )
  # remove numbers
  number_removed = tf.strings.regex_replace(
      email_removed, "[0-9]", ' '
  )
  tab_removed = tf.strings.regex_replace(number_removed, "\t+", " ")
  # remove punctuations
  punctuation_removed =  tf.strings.regex_replace(
      tab_removed, '[%s]' % re.escape(string.punctuation), ' '
  )
  # remove multiple blank spaces
  # multiple_space_removed =tf.strings.reduce_join(tf.strings.split(punctuation_removed), separator=" ")
  multiple_space_removed = tf.strings.regex_replace(punctuation_removed, ' +', ' ')
  return multiple_space_removed


In [78]:
vocab_size = len(set(total_words))
sequence_length = 10

vectorize_layers = layers.TextVectorization(
    standardize=preprocess,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length
)

In [79]:
vectorize_layers.adapt(train_data.batch(1024))
# vectorize_layers.adapt(train_words)

In [80]:
print(vectorize_layers.vocabulary_size())
print(vectorize_layers.get_vocabulary()[:20])

77710
['', '[UNK]', 'the', 'to', 'of', 'a', 'ax', 'and', 'i', 'in', 'is', 'that', 'it', 'for', 'you', 's', 'from', 't', 'on', 'this']


In [81]:
inverse_vocab = vectorize_layers.get_vocabulary()
inverse_vocab[-20:]

['aangezien',
 'aangegeven',
 'aangeboden',
 'aanerud',
 'aanbieden',
 'aams',
 'aamazing',
 'aam',
 'aalternate',
 'aah',
 'aaef',
 'aacc',
 'aaahh',
 'aaaaagggghhhh',
 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaauuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuugggggggggggggggg',
 'aaaaaaaaaaaa',
 '\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e',
 '\x18',
 '\x08s',
 '\x08r']

In [82]:
text_vector_ds = train_data.batch(1024).prefetch(AUTOTUNE).map(vectorize_layers).unbatch()

In [83]:
# Sub sampling
for i in text_vector_ds.take(2):
  print(f"Vector: {i.numpy()} => word: {[inverse_vocab[x] for x in i]}")

Vector: [  16 3558    0    0    0    0    0    0    0    0] => word: ['from', 'mathew', '', '', '', '', '', '', '', '']
Vector: [  31 1032 1070  503 1597 1480    0    0    0    0] => word: ['subject', 'alt', 'atheism', 'faq', 'atheist', 'resources', '', '', '', '']


In [123]:
print(inverse_vocab[3558])

if inverse_vocab[3558] in subsampled_words:
  print("True")
else:
  print("False")

mathew
True


In [84]:
sequences = list(train_data.as_numpy_iterator())
print(len(sequences))

511617


In [85]:
if inverse_vocab[-1] in subsampled_words:
  print("true")

true


### Creating vocab

In [21]:
# vocab = Counter(set(total_words))
id_word = {id:word for id, word in enumerate(inverse_vocab)}
word_id = {word:id for id, word in id_word.items()}

### Skip-gram with sub-sampling

In [128]:
def skip_gram(index_vocab, context_window):
  context_vector = []
  # negative_vector = []
  try:
    padding_index = index_vocab.tolist().index(0)
    for index, target in enumerate(index_vocab):
      # context = index_vocab[ index - context_window between index + context_window ] except index
      # TODO
      # check if the target's probability < threshold
      if (index<padding_index) and (inverse_vocab[target] in subsampled_words):
        left_index = index-context_window if (index-context_window) >= 0 else 0
        right_index = index+context_window+1 if (index+context_window) < padding_index else padding_index
        context_vector.extend([[index_vocab[index], index_vocab[c]] for c in range(left_index, right_index) if (c != index and inverse_vocab[index_vocab[c]] in subsampled_words)])
      # elif index > padding_index-context_window-1 and index < padding_index:
        # left_index = index-context_window if (index-context_window) >= 0 else 0
        # right_index = index+context_window+1 if (index+context_window+1) <= padding_index else len(padding_index)
        # context_vector.extend([[index_vocab[index], index_vocab[c]] for c in range(left_index, right_index) if c != index])
      
      else:
        continue
  except:
    for index, target in enumerate(index_vocab):
    # context = index_vocab[ index - context_window between index + context_window ] except index
    # TODO
    # Check probability i.e sub-sampling need to be done
      if (inverse_vocab[target] in subsampled_words):
          left_index = index-context_window if (index-context_window) >= 0 else 0
          right_index = index+context_window+1 if (index+context_window) < len(index_vocab) else len(index_vocab)
          context_vector.extend([[index_vocab[index], index_vocab[c]] for c in range(left_index, right_index) if (c != index and inverse_vocab[index_vocab[c]] in subsampled_words)])
      else:
        continue

  context_vector.sort()
  context_vector = list(k for k, _ in itertools.groupby(context_vector))
  return context_vector

  #   negative_sample = negative_sampling(target, context_vector, negative_sample_number, vocab, vocab_size)
  #   negative_vector.extend(negative_sample)
  # return context_vector, negative_vector

In [129]:
example_sentence = [31, 1032, 1070, 503, 1597, 1480]
window_size=2
positive_skip_grams_2 = skip_gram(example_sentence, window_size)
print(positive_skip_grams_2)
print(len(positive_skip_grams_2))

[[503, 1032], [503, 1070], [503, 1480], [503, 1597], [1032, 503], [1032, 1070], [1070, 503], [1070, 1032], [1070, 1597], [1480, 503], [1480, 1597], [1597, 503], [1597, 1070], [1597, 1480]]
14


In [131]:
if inverse_vocab[31] in subsampled_words:
  print("True")

In [95]:
for i in example_sentence:
  if inverse_vocab[31] in subsampled_words:
    print("True")

### Negative sampling

In [132]:
## negative sampling
def negative_sampling(target, context, negative_sample_number, vocab_size):
  # vocab_size = len(vocab)
  # TODO
# this implementation continues in loop when context_window is greater
# suppose we have vocab size of 4 [1,2,3,4] and context_window = 3 and negative sample = 2
# then we can not get a negative sample of size 2 and the while loop will continue forever
# this need to be handle
  negative_sample = []
  # negative_sample_number = 2
  # vocab_size = 7
  # vocab = index_vocab
  # or get the context for particular index and sample negative values from vocab except the context
  while len(negative_sample) < negative_sample_number:
    import pdb
    # pdb.set_trace()
    sample = random.randint(0, vocab_size-1)
    if (sample != context) and (sample not in negative_sample) and sample != target:
      negative_sample.append(sample)
  return tf.convert_to_tensor(negative_sample, dtype='int64')

### Generate training data

In [133]:
def skip_gram_with_negative_sampling(sequences,context_window, negative_sample_number, vocab_size):
  targets, contexts, labels = [],[],[]
  for sequence in tqdm.tqdm(sequences):
    # import pdb
    positive_skip_grams = skip_gram(sequence, context_window)
    # pdb.set_trace()
    if positive_skip_grams:
      for target_word, context_word in positive_skip_grams:
        context_class = tf.expand_dims(tf.constant([context_word], dtype='int64'), 1)
        negative_sampling_candidate = negative_sampling(target_word, context_word, negative_sample_number, vocab_size)

        negative_sampling_candidate = tf.expand_dims(negative_sampling_candidate, 1)
        context = tf.concat([context_class, negative_sampling_candidate], 0)
        label = tf.constant([1] + [0]*negative_sample_number, dtype="int64")

        targets.append(target_word)
        contexts.append(context)
        labels.append(label)
  
  return targets, contexts, labels


In [ ]:
# Mapping sampled training words to int

training_word_int = [word_id[word] for word in train_words]
training_word_int[:10]

NameError: ignored

In [ ]:
targets, contexts, labels = skip_gram_with_negative_sampling(
    sequences=sequences,
    context_window=2,
    negative_sample_number=4,
    vocab_size=vocab_size,
)

targets = np.array(targets)
contexts = np.array(contexts)[:,:,0]
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

  0%|          | 2073/511617 [59:13<202:42:57,  1.43s/it]

In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

In [ ]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

### Model

In [ ]:
num_ns = 4
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                             embedding_dim,
                                             input_length=1,
                                             name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                              embedding_dim,
                                              input_length=num_ns+1)
    
  def call(self, pair):
    target, context = pair
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    
    word_emb = self.target_embedding(target)
    context_emb = self.context_embedding(context)
    dots = tf.einsum('be, bce->bc', word_emb, context_emb)
    return dots

### Training

In [ ]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [ ]:
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

In [ ]:
%tensorboard --logdir logs